In [139]:
import taxcalc as tc
import pandas as pd
import numpy as np
import copy
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook

In [140]:
output_notebook()

Loading BokehJS ...

In [141]:
def taxdf(calc, namelist, varlist):
    calcu=copy.deepcopy(calc)
    tdf=pd.DataFrame()
    for v in range (0, len(namelist)):
        tdf[namelist[v]]=getattr(calcu.records,varlist[v])
    return tdf

In [142]:
def createcalc(year, records=tc.Records(), policy=tc.Policy()):
        calc=tc.Calculator(records=records, policy=policy)
        calc.advance_to_year(year)
        calc.calc_all()
        return calc

In [143]:
def w_avg(df, var):
    return (df[var] * (df['s006'] / df['s006'].sum())).sum()

In [144]:
vlist=['c00100', 'standard', 'c04470', 'iitax','c19700', 'c17000','e17500', 'e19800','e20100','combined','MARS','c18300','s006']

In [145]:
# 2017 Reform
path_name = 'C:\\Users\\derrick.choe\\Tax-Calculator\\taxcalc\\reforms\\2017_law.json'
jfile = open(path_name, 'r')
jtext = jfile.read()
jfile.close()
reform2017 = tc.Calculator._read_json_policy_reform_text(jtext, {}, {})

pol2017 = tc.Policy()
pol2017.implement_reform(reform2017)

#Baseline Calculator
calc_2017=createcalc(year = 2017, records = tc.Records(), policy = pol2017)

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


# Taxpayer Deciles

In [161]:
def groupfig(df, var, bins, title = ""):
    graphlist = []
    for b in range(df['bins'].min(), df['bins'].max() + 1):
        subpop = copy.deepcopy(df[df['bins'] == b])
        graphlist.append(w_avg(subpop, var))
    f = figure(title = title)
    f.xaxis.axis_label = 'AGI Bins: ' + str(df['bins'].min()) + " through " + str(df['bins'].max())
    f.line(range(df['bins'].min(), df['bins'].max() + 1), graphlist)
    show(f)
    return graphlist

In [162]:
mydf = taxdf(calc_2017, vlist, vlist)
tc.add_quantile_bins(mydf, income_measure='c00100', num_bins = 10)
mydf['deduct_medical'] = np.where(mydf['c17000'] > 0, 1, 0)
groupfig(mydf, 'deduct_medical', 10, 'Fraction Who Take Medical Deduction')

[0.007932709778616362,
 0.027830570037835827,
 0.038547878458746819,
 0.04699350345661181,
 0.06571292565537333,
 0.072064184687356841,
 0.093979730032202877,
 0.094385032172553215,
 0.065489622401072242,
 0.024961825899324198]

In [163]:
mydf_c = copy.deepcopy(mydf[mydf['bins'] > 2])
mydf_c['frac_med'] = np.where((mydf_c['c17000'] > 0) & (mydf_c['c00100'] != 0) , mydf_c['e17500'] / mydf_c['c00100'], np.nan)
mydf_c['s006'] = np.where((mydf_c['c17000'] > 0) & (mydf_c['c00100'] != 0) , mydf_c['s006'], 0)
groupfig(mydf_c, 'frac_med', 10, 'Medical Expenses as Fraction of AGI: Population Taking Medical Deduction')

[1.1754890024508562,
 0.66652942864108711,
 0.50323809504698014,
 0.34410933226668766,
 0.27571703218902033,
 0.21616547752544779,
 0.17972604118507612,
 0.18752581905747884]